<h1> Introdução </h1>

Este código aborda um problema de agendamento de admissão de pacientes em camas de hospital, considerando as restrições relacionadas com o oxigénio, a telemetria e a disponibilidade de camas em quartos específicos. O objetivo é encontrar uma atribuição de camas para os pacientes ao longo dos seus períodos de internamento, respeitando as restrições mencionadas.

Este trabalho foi desenvolvido por:

* a13725 – Alexandre Manuel Oliveira de Castro Pereira
* a24003 – Daniel da Silva Pires
* a19631 – Tiago José Gomes Morais


<h2> Formulação do problema </h2>

**O objetivo é encontrar uma solução para o problema de agendamento de admissão dos pacientes em camas de hospital, levando em consideração as seguintes restrições:**

* Dois pacientes não podem ocupar a mesma cama no mesmo dia.

* Pacientes que precisam de oxigénio devem ser colocados em camas que pertencem a quartos com oxigénio.

* Pacientes que precisam de telemetria devem ser colocados em camas que pertencem a quartos com telemetria.

<h2> Plano e Design de um Agente Adequado </h2>

**Agente (PEAS):** Sistema de Agendamento de Admissão de Pacientes

**Medida de Desempenho:** eficiência na colocação de pacientes nas camas, maximizando a utilização dos recursos, minimizando conflitos de restrições médicas e temporais, e otimizando a satisfação do paciente.


**Ambiente:**
O ambiente é o hospital, com uma série de camas, cada uma associada a um quarto com condições específicas de oxigénio e telemetria.

* O ambiente é totalmente observável, pois o agente tem conhecimento completo sobre o estado de calendarização de internamentos do hospital.

* O ambiente é sequencial, pois as ações do agente impactam as futuras decisões. Cada colocação é um episódio.

* O ambiente é dinâmico devido às admissões e altas dos pacientes.

* O ambiente é discreto, com a colocação de pacientes sendo uma ação discreta num conjunto finito de camas e pacientes.

* O ambiente é determinístico, pois as ações do agente determinam diretamente as consequências no ambiente.

**Sensor:** Os sensores são representados pelos dados fornecidos ao agente, como as condições médicas dos pacientes, a disponibilidade das camas e as restrições temporais.

**Atuador:** O atuador é representado pela parte do código que implementa a colocação de pacientes nas camas.


<h3> Formulação do Problema como um Problema de Procura: </h3>

O problema é formulado como um problema de procura para encontrar uma atribuição válida de camas para os pacientes ao longo dos seus períodos de internamento, respeitando as restrições. Cada paciente é associado a uma variável que representa a cama atribuída durante os dias de internamento.

<h2> Agente em Execução </h2>

**Solução para um ou mais Estados Iniciais:**

A execução do agente no sistema de agendamento de admissão de pacientes resulta numa colocação de camas para os pacientes, otimizando as condições médicas e temporais respeitando as restrições de sobreposição de datas, necessidade de oxigénio e telemetria.

**Exemplo de Solução:**

```bash
Paciente 1 - Cama 3 durante os dias {0}
Paciente 2 - Cama 4 durante os dias {0, 1, 2, 3, 4, 5}
Paciente 3 - Cama 6 durante os dias {0, 1}
Paciente 4 - Cama 7 durante os dias {0, 1, 2, 3, 4}
Paciente 5 - Cama 8 durante os dias {0, 1, 2, 3}
Paciente 6 - Cama 5 durante os dias {0, 1}
Paciente 7 - Cama 2 durante os dias {1, 2, 3, 4, 5, 6, 7}
Paciente 8 - Cama 5 durante os dias {2, 3}
Paciente 9 - Cama 3 durante os dias {2, 3, 4, 5}
Paciente 10 - Cama 6 durante os dias {3, 4, 5, 6, 7, 8, 9, 10}
Paciente 11 - Cama 8 durante os dias {4, 5, 6, 7, 8, 9, 10}
Paciente 12 - Cama 5 durante os dias {5, 6, 7, 8, 9, 10, 11}
Paciente 13 - Cama 7 durante os dias {5, 6, 7, 8, 9, 10, 11, 12}
Paciente 14 - Cama 3 durante os dias {7, 8, 9, 10, 11, 12, 13}
Paciente 15 - Cama 4 durante os dias {8, 9}
Paciente 16 - Cama 2 durante os dias {9, 10, 11, 12, 13, 14, 15}
Paciente 17 - Cama 4 durante os dias {10, 11, 12, 13, 14, 15, 16, 17}
Paciente 18 - Cama 1 durante os dias {10, 11, 12, 13, 14}
Paciente 19 - Cama 6 durante os dias {11, 12, 13, 14, 15, 16, 17}
Paciente 20 - Cama 8 durante os dias {12, 13, 14, 15, 16, 17, 18, 19}
```

<h3> Conclusão </h3>

Ao longo do projeto deparamo-nos com algumas dificuldades quanto à abordagem do problema. Começamos com uma abordagem quanto ás variáveis que iríamos utilizar, que mais tarde vimos não ser a melhor abordagem e por isso estarmos a ter dificuldades na continuação da solução pretendida. 
Superados estes desafios, o projeto avançou a um bom ritmo e conseguimos concluir a solução a que nos proposemos.

Apesar de nem todos os elementos do grupo estarem tão à vontade com a tecnologia utilizada, conseguimos agilizar entre nós de modo a que o projeto avançasse a bom ritmo e nenhum elemento do grupo ficou à parte quanto ao funcionamento e lógica por trás da solução implementada.

Em suma, apesar das dificuldades, conseguimos alcançar a solução que nos proposemos realizar e estamos satisfeitos com os resultados do nosso esforço. Cremos que a solução alcançada, corresponde ao objetivo do problema proposto e com um bom desempenho no que toca ao tempo para adquirir uma solução. Sabemos que existe margem para melhoria, nomeadamente, a implementação de mais restrições ao problema e talvez a atribuição de um peso nas soluções por forma a apresentar a solução ótima ao invés da primeira solução alcançada.

# Codigo

Importar as bibliotecas necessárias

In [41]:
import ast
from constraint import Problem
from dataclasses import dataclass
from enum import Enum

Criação das classes

In [42]:
@dataclass
class Room:
    id: int
    oxygen: bool
    telemetry: bool


@dataclass
class Bed:
    id: int
    room: Room

    def __repr__(self) -> str:
        return str(self.id)


class Gender(Enum):
    MALE = ("MALE",)
    FEMALE = ("FEMALE",)
    OTHER = "OTHER"


@dataclass
class Patient:
    id: int
    name: str
    age: int
    gender: Gender
    admission_date: int
    discharge_date: int
    oxygen: bool
    telemetry: bool

Funções auxiliares

In [ ]:
def find_patients(patients, id):
    for patient in patients:
        if patient.id == id:
            return patient
    return None


def get_bed_by_id(beds, bed_id) -> Bed:
    for bed in beds:
        if bed.id == bed_id:
            return bed
    return None

Setup dos dados

In [ ]:
problem = Problem()

rooms = [
    Room(1, True, False),
    Room(2, True, True),
    Room(3, True, False),
    Room(3, True, True),
]

beds = [
    Bed(1, rooms[0]),
    Bed(2, rooms[0]),
    Bed(3, rooms[1]),
    Bed(4, rooms[1]),
    Bed(5, rooms[2]),
    Bed(6, rooms[2]),
    Bed(7, rooms[3]),
    Bed(8, rooms[3]),
]

patients = [
    Patient(1, "Patient1", 98, Gender.MALE, 0, 0, False, False),
    Patient(2, "Patient2", 82, Gender.MALE, 0, 5, True, True),
    Patient(3, "Patient3", 43, Gender.MALE, 0, 1, False, False),
    Patient(4, "Patient4", 88, Gender.MALE, 0, 4, False, False),
    Patient(5, "Patient5", 20, Gender.FEMALE, 0, 3, False, True),
    Patient(6, "Patient6", 65, Gender.FEMALE, 0, 1, False, False),
    Patient(7, "Patient7", 33, Gender.FEMALE, 1, 7, True, False),
    Patient(8, "Patient8", 86, Gender.MALE, 2, 3, False, False),
    Patient(9, "Patient9", 22, Gender.FEMALE, 2, 5, False, True),
    Patient(10, "Patient10", 70, Gender.FEMALE, 3, 10, True, False),
    Patient(11, "Patient11", 42, Gender.MALE, 4, 10, True, True),
    Patient(12, "Patient12", 3, Gender.FEMALE, 5, 11, False, False),
    Patient(13, "Patient13", 14, Gender.FEMALE, 5, 12, False, True),
    Patient(14, "Patient14", 78, Gender.MALE, 7, 13, False, False),
    Patient(15, "Patient15", 29, Gender.FEMALE, 8, 9, True, False),
    Patient(16, "Patient16", 61, Gender.FEMALE, 9, 15, False, False),
    Patient(17, "Patient17", 56, Gender.FEMALE, 10, 17, False, True),
    Patient(18, "Patient18", 106, Gender.FEMALE, 10, 14, True, False),
    Patient(19, "Patient19", 4, Gender.MALE, 11, 17, True, False),
    Patient(20, "Patient20", 52, Gender.FEMALE, 12, 19, True, True),
]

Adicionar as variáveis com o seu domínio de dias

In [ ]:
for patient in patients:
        days_for_patient = set(
            range(patient.admission_date, patient.discharge_date + 1)
        )
        domain = [f"bed_{bed}_days_{days_for_patient}" for bed in beds]
        problem.addVariable(f"patient_{patient.id}", domain)

Avalia para cada paciente se o número de dias (set) interceta com outro paciente,
Se sim, a condição é falsa excepto se a cama for diferente

In [ ]:
for variable, domain in problem._variables.items():
    patient = variable.split("_")[1]
    patients_except_var = [p for p in problem._variables if p != variable]

    def test(a, b):
        bed_a = a.split("_")[1]
        bed_b = b.split("_")[1]
        set_a = ast.literal_eval(a.split("_")[3])
        set_b = ast.literal_eval(b.split("_")[3])
        return len(set_a & set_b) == 0 or bed_a != bed_b

    for patient in patients_except_var:
        problem.addConstraint(test, [variable, patient])

Adicionar constraint de oxigénio e telemetria

In [ ]:
for variable, domain in problem._variables.items():
    patient_id = int(variable.split("_")[1])
    patient = find_patients(patients, patient_id)

    if patient.oxygen == True:

        def test_oxygen(a):
            bed_id = int(a.split("_")[1])
            bed = get_bed_by_id(beds, bed_id)
            return bed.room.oxygen

        problem.addConstraint(test_oxygen, [variable])

    if patient.telemetry == True:

        def test_telemetry(a):
            bed_id = int(a.split("_")[1])
            bed = get_bed_by_id(beds, bed_id)
            return bed.room.telemetry

        problem.addConstraint(test_telemetry, [variable])

Correr o CSP

In [ ]:
solutions = dict(problem.getSolution())
solutions = {int(k.split("_")[1]):v for (k, v) in solutions.items()}
solutions = dict(sorted(solutions.items()))


Imprimir as soluções

In [44]:
if solutions:
    for k, v in solutions.items():
            print(f"Paciente {k} - Cama {v.split('_')[1]} durante os dias {v.split('_')[3]}")
else:
    print("No solution found.")

 Paciente 1 - Cama 3 durante os dias {0}
 Paciente 2 - Cama 4 durante os dias {0, 1, 2, 3, 4, 5}
 Paciente 3 - Cama 6 durante os dias {0, 1}
 Paciente 4 - Cama 7 durante os dias {0, 1, 2, 3, 4}
 Paciente 5 - Cama 8 durante os dias {0, 1, 2, 3}
 Paciente 6 - Cama 5 durante os dias {0, 1}
 Paciente 7 - Cama 2 durante os dias {1, 2, 3, 4, 5, 6, 7}
 Paciente 8 - Cama 5 durante os dias {2, 3}
 Paciente 9 - Cama 3 durante os dias {2, 3, 4, 5}
 Paciente 10 - Cama 6 durante os dias {3, 4, 5, 6, 7, 8, 9, 10}
 Paciente 11 - Cama 8 durante os dias {4, 5, 6, 7, 8, 9, 10}
 Paciente 12 - Cama 5 durante os dias {5, 6, 7, 8, 9, 10, 11}
 Paciente 13 - Cama 7 durante os dias {5, 6, 7, 8, 9, 10, 11, 12}
 Paciente 14 - Cama 3 durante os dias {7, 8, 9, 10, 11, 12, 13}
 Paciente 15 - Cama 4 durante os dias {8, 9}
 Paciente 16 - Cama 2 durante os dias {9, 10, 11, 12, 13, 14, 15}
 Paciente 17 - Cama 4 durante os dias {10, 11, 12, 13, 14, 15, 16, 17}
 Paciente 18 - Cama 1 durante os dias {10, 11, 12, 13, 14}
 